Disclaimer: This notebook is based and reproduces information from chapter 10 of the book [An Introduction to Statistical Learning with Applications in R](http://www-bcf.usc.edu/~gareth/ISL/) written by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. All rights of the material belong to the authors.

# PRINCIPAL COMPONENT ANALYSIS (PCA)

- PCA is an **unsupervised approach**:
    
    - We have only a set of features measured on n observations, but no response Y
    
    - This means we cannot easily assess the accuracy of the results obtained 


- PCA finds a **low-dimensional representation** of the data that captures as much of the information as possible

    - Reduces an n x p data matrix by reducing the size of p (variables, also called features)


- Principal component analysis (PCA) refers to the process by which **principal components** are computed

### What is a principal component?

- The first principal component direction of the data is that along which the observations vary the most.


- Each of the component found by PCA is a **linear combination of the p features**:

    ![image_linear_combination.png](./images/linear_combination_pca.png)


#### Loading vectors

- The elements φ11, . . . , φp1 are the loadings of the first principal component

- Together, the loadings make up the principal component loading vector:

    ![image_loading_vectors.png](./images/loading_vectors.png)



- The loading vectors are normalized -> their sum of squares is equal to 1

    ![image_normalized_loading_vectors.png](./images/normalized_loading_vectors.png)


#### Principal component scores
- The values of z11, . . . , zn1 are the principal component scores


#### Geometric interpretation
- Principal component loading vectors -> directions in feature space along which the data vary the most


- Principal component scores -> projections along these direction


#### Alternative interpretation of the principal components 
- The first principal component vector defines the line that is as close as possible to the original observations 
    
    - It minimizes the sum of squared distances from each point to the plane

        ![image_pca_closeness.png](./images/pca_closeness.png)


### How do we compute the first principal component?

1. Since we are only interested in variance, we assume that each of the variables in X has been centered to have mean zero 


2. We then look for the linear combination of the sample feature values that has the largest variance, which solves the optimization problem: 

    ![image_optimization_pca.png](./images/optimization_pca.png)


3. We then look for the second principal component, which is the linear combination of X1, . . . , Xp that has maximal variance out of all linear combinations that are uncorrelated/orthogonal/perpendicular with Z1



Let's use the dataset USA-ARRESTS to put into practice these concepts:

## Load and prepare the dataset

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Let's import and print the USA arrest data:

In [ ]:
# Load and print USA arrest data
usarrest_df = pd.read_csv('../data/usa_arrest.csv')
print(usarrest_df)

For each of the 50 states in the United States, the data set contains the number of arrests per 100, 000 residents for each of three crimes: Assault, Murder, and Rape. It also contains the percent of the population in each state living in urban areas.

Notice that the variables are measured in different units.

Let's explore the mean, variance and standard deviation of each variable (feature):

In [ ]:
# Print mean value of each feature
usarrest_df.mean()

In [ ]:
# Print variance value of each feature
usarrest_df.var()

In [ ]:
# Print std of each feature
usarrest_df.std()

We can observe that features differ greatly in their mean, variance and standard deviation. 

This is information to keep in mind when performing principal component analysis (PCA).

Next, let's put the data into the right formats for performing PCA. We will begin by creating separate arrays containing the values of the observations and the features:

In [ ]:
# Create array of states
states = usarrest_df.index.get_values()

# Create arrays of features
features = usarrest_df.columns.get_values()

We will also convert the whole dataset to numpy array to give it as input to scikit-learn:

In [ ]:
# Convert data to numpy array
usarrest = usarrest_df.loc[:, features].values

- The results obtained when we perform PCA will depend on whether the variables have been individually scaled (each multiplied by a different constant)


- In our case, if we do not scale the data, the principal component loading vector will have a very large loading for Assault, since that variable has by far the highest variance.


Let's convert the data to have mean 0 and std 1, since feature's values are in different scale:

In [ ]:
# Standarize dataset
scaler = StandardScaler()
usarrest_scaled = scaler.fit_transform(usarrest)

## Perform PCA

Finally, let's perform PCA over the scaled data:

In [ ]:
# Perform PCA
pca = PCA()
pca_score_vectors = pca.fit_transform(usarrest_scaled)   # store score vectors in new variable

Let's examine each PCA loading vector:

In [ ]:
# Print PCA loadings
columns_pca = ['PCA_' + str(pca +1) for pca in range(0, np.shape(pca_score_vectors)[1])]
pca_loadings = (pca.components_).T
pca_loadings_df = pd.DataFrame(pca_loadings,
                               index=features, 
                               columns=columns_pca)
print(pca_loadings_df)

Let's examine each PCA score vector:

In [ ]:
# Print principal component score vectors
pca_score_vectors_df = pd.DataFrame(pca_score_vectors, 
                                    index=states, 
                                    columns=columns_pca)
print(pca_score_vectors_df)

As a sanity check, let's make sure that the product of our scaled dataset and the loadings of the first component returns the score vectors that we obtained for the first component:

In [ ]:
# Compute dot product between X and loadings of the first component
pca1_score_vectors = np.dot(usarrest_scaled, pca_loadings[:,0])
print(pca1_score_vectors)

We observe that our vector is the same as that returned by scikit-learn

## Plot PCA results

- A biplot is a useful way of visualizing the results of PCA


- It displays both the principal component scores and the principal component loadings

Let's make a biplot of the first two components:

In [ ]:
# Biplot of the first two components
fig, ax1 = plt.subplots(figsize=(7, 7));

rect = 0, 0, 1, 1  

# plot score points
ax1 = fig.add_axes(rect)

ax1.scatter(pca_score_vectors[:,0], pca_score_vectors[:,1],
           alpha=0.3)
for i in range(pca_score_vectors.shape[0]):
    ax1.text(pca_score_vectors[i,0], pca_score_vectors[i,1], states[i],
             alpha=0.8)

min_sv = np.min(pca_score_vectors[:,0:2])
max_sv = np.max(pca_score_vectors[:,0:2])
ax1.set_xlim(min_sv - 0.5, max_sv + 0.5)
ax1.set_ylim(min_sv - 0.5, max_sv + 0.5)

ax1.set_xlabel('PCA_1')
ax1.set_ylabel('PCA_2')

# plot loading vectors
ax2 = fig.add_axes(rect,  frameon=False)

for i in range(pca_loadings.shape[0]):
    ax2.arrow(0, 0, pca_loadings[i,0], pca_loadings[i,1], 
              color = 'r', alpha = 1,
              head_width=0.02, head_length=0.02)
    ax2.text(pca_loadings[i,0]* 1.2,  pca_loadings[i,1] * 1.1, features[i], 
             color = 'r', ha = 'center', va = 'center')
    
ax2.set_xlim(-1,1)
ax2.set_ylim(-1,1)

ax2.yaxis.tick_right()
ax2.yaxis.set_ticks([-0.5, 0, 0.5])
ax2.yaxis.set_label_position('right')
ax2.xaxis.tick_top()
ax2.xaxis.set_ticks([-0.5, 0, 0.5])
ax2.xaxis.set_label_position('top')

ax2.grid()    


In chapter 10, the authors intepret from the plot the following: 

- The first loading vector places approximately equal weight on Assault, Murder, and Rape, with much less weight on UrbanPop. Hence this component roughly corresponds to a measure of overall rates of serious crimes. 

- The second loading vector places most of its weight on UrbanPop and much less weight on the other three features. Hence, this component roughly corresponds to the level of urbanization of the state.

- Overall, we see that the crime-related variables (Murder, Assault, and Rape) are located close to each other, and that the UrbanPop variable is far from the other three. This indicates that the crime-related variables are correlated with each other—states with high murder rates tend to have high assault and rape rates—and that the UrbanPop variable is less correlated with the other three.


- States with large positive scores on the first component, such as California, Nevada and Florida, have high crime rates, while states like North Dakota, with negative scores on the first component, have low crime rates. 

- California also has a high score on the second component, indicating a high level of urbanization, while the opposite is true for states like Mississippi. 

- States close to zero on both components, such as Indiana, have approximately average levels of both crime and urbanization.

## Compute the proportion of variance explained by each component

- Important to know how much of the information in a given data set is lost by projecting the observations onto the first few principal components


- This can be answered by knowing the proportion of variance explained (PVE) by each component.


- The proportion of the variance explained by the mth component is the variance explained by the mth component divided by the total variance of the dataset 

- The variance explained by the _m_th component is defined as:

    ![image_variance_component.png](../images/variance_component.png)
    
- The total variance of a dataset is defined as:

    ![image_total_variance.png](../images/total_variance.png)

- Therefore, the PVE is given by: 

    ![image_PVE.png](../images/PVE.png)


Let's compute the variance explained, and the proportion of variance explained, of each component with sci-kit learn:

In [ ]:
# Print explained variance by each component
variance = pca.explained_variance_
variance_ratio = pca.explained_variance_ratio_

variance_df = pd.DataFrame([variance, variance_ratio], 
                           index = ['Variance', 'Variance_ratio'], 
                           columns=columns_pca)
print(variance_df)

### Plot PVE's

- We typically decide on the number of principal components required to visualize the data by examining a scree plot. 


- This is done by eyeballing the scree plot, and looking for a point at which the proportion of variance explained by each subsequent principal component drops off (often refered as the elbow in the scree plot)


Let's plot the PVEs:

In [ ]:
# Plot variance explained by each component 
plt.plot(variance_df.loc['Variance_ratio'])
plt.title('Proportion of variance explained by each component')
plt.ylabel('Variance Ratio');
plt.xlabel('Component');

### Cumulative sum of PVE's

We can also decide to examine the cumulative sum of PVE's:

In [ ]:
# Plot cumulative sum of variance explained by each component
cumulative_values = np.cumsum(variance_ratio)

plt.plot(cumulative_values)
plt.title('Cumulative proportion of variance explained')
plt.ylabel('Variance Ratio');
plt.xlabel('Component');

## Other uses of PCA

- Apart from producing derived variables for use in supervised learning problems, PCA also serves as a tool for data visualization

- We can also use the principal component score vectors as features for other analyses, such as regression, classification, and clustering (using as input the n × M matrix) 